In [1]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder

In [2]:
t = torch.cuda.get_device_properties(0).total_memory
print('Total Memory: {}'.format(t / 1e9))

Total Memory: 17.071734784


In [3]:
!pip install simpletransformers

     |████████████████████████████████| 194kB 4.7MB/s 
     |████████████████████████████████| 778kB 13.2MB/s 
     |████████████████████████████████| 317kB 13.7MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 3.0MB 23.9MB/s 
     |████████████████████████████████| 890kB 43.2MB/s 
     |████████████████████████████████| 1.1MB 51.8MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=3a9d178fae1b1b57b856119ba5882513ebdae4d4bb028160d2c8103cdc973624
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=48ca31f112ab8e8cb65948a4397d446acb158f2d2dedd31ad45d8467f504f724
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses
ERROR: transformers 3.0.2 has requirement tok

In [ ]:
# !pip install transformers==2.10.0
# !pip install simpletransformers==0.26.0

In [4]:
from simpletransformers.classification import ClassificationModel

In [5]:
le = LabelEncoder()

In [14]:
# train = pd.read_csv('drive/My Drive/data-class/train.csv', names=['text', 'Code', 'country'], header=0)
train = pd.read_csv('drive/My Drive/data-class/train_translated.csv', names=['text', 'Code'], header=0)
# train['tmp'] = train.Code.apply(lambda x: int(x) // 100)
# train["labels"] = le.fit_transform(train.tmp)
train["labels"] = le.fit_transform(train.Code)
train_df = train[["text", "labels"]]
train_df.head(5)

,text,labels
0,"In questo senso, una maggiore flessibilità del...",33
1,"En este sentido, una mayor flexibilidad del me...",33
2,"Nesse sentido, uma maior flexibilidade do merc...",33
3,- la depenalizzazione della coltivazione perso...,36
4,- la despenalización del cultivo personal de c...,36


In [15]:
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [16]:
train_df.head()

,text,labels
0,González ribalta il suo attacco alla Corte Sup...,37
1,Nuestro objetivo es implementar la ciudadanía ...,8
2,Los ‘indignados’ cruzan la ‘línea roja’ y agre...,12
3,- In situazioni di mancanza o rottura della re...,34
4,A queda do império mobutista,14


In [17]:
val = pd.read_csv('drive/My Drive/data-class/val.csv', names=['text', 'Code', 'country'], header=0)
# val['tmp'] = val.Code.apply(lambda x: int(x) // 100)
# val["labels"] = le.fit_transform(val.tmp)
val["labels"] = le.fit_transform(val.Code)
val_df = val[["text", "labels"]]
val_df.head(5)

,text,labels
0,Parte significativa do orçamento doméstico é c...,43
1,Clínicas de reproducción pagan 750 E a univers...,33
2,Ceuta levanta ya otra valla para frenar las en...,39
3,España exige a Rabat que frene el tréfico de d...,5
4,"Eguiguren: ""ETA perdería contra Batasuna""",37


In [18]:
test = pd.read_csv('drive/My Drive/data-class/test.csv', names=['text', 'Code', 'country'], header=0)
# test['tmp'] = test.Code.apply(lambda x: int(x) // 100)
# test["labels"] = le.fit_transform(test.tmp)
test["labels"] = le.fit_transform(test.Code)
test_df = test[["text", "labels"]]
test_df.head(5)

,text,labels
0,La ministra colombiana de Exteriores considera...,37
1,DEPORTES,31
2,É enorme a importância dessas atividades no de...,31
3,"Com o fim da guerra fria, iniciou-se um novo p...",5
4,Revelaram-se com inapagável nitidez novas teia...,13


In [20]:
model = ClassificationModel(
                            'xlmroberta',
                            #'xlm-roberta-base',
                            #'bert',
                            #'drive/My Drive/roberta-class-lm/',
                            #'bert-base-multilingual-cased', 
                            #'drive/My Drive/bert-lm/',
                            'drive/My Drive/roberta-lm/',
                            num_labels=len(train_df.labels.unique()), 
                            args={'train_batch_size': 32, 
                                  'eval_batch_size': 32,
                                  'num_train_epochs': 4,
                                  'max_seq_length': 200,
                                  'weight_decay': 0.01,
                                  'adam_epsilon': 1e-6,
                                  'warmup_ratio': 0.1,
                                  'evaluate_during_training': True,
                                  'evaluate_during_training_verbose': True,
                                  'save_model_every_epoch': False,
                                  'save_eval_checkpoints': False,
                                  'save_steps': 0,
                                  'use_early_stopping': True,
                                  'early_stopping_patience': 2,
                                  'fp16': False,
                                  'overwrite_output_dir': True,
                                  'reprocess_input_data': True,
                                  'learning_rate': 2e-5,
                                  'manual_seed':42
                                  }) 


Some weights of the model checkpoint at drive/My Drive/roberta-lm/ were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at drive/My Drive/roberta-lm/ and are newly initialized: ['classifier.dense.weight', 'class

In [ ]:
model.train_model(train_df, eval_df=val_df)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
from sklearn import metrics

In [ ]:
def f_macro(true, preds):
  return metrics.f1_score(true, preds, average='macro')

def f_micro(true, preds):
  return metrics.f1_score(true, preds, average='micro')

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=metrics.accuracy_score, f1M=f_macro, f1m=f_micro)

In [ ]:
result

{'acc': 0.6231697665215671,
 'eval_loss': 1.4780973090086809,
 'f1M': 0.4606981569218518,
 'f1m': 0.6231697665215671,
 'mcc': 0.6006091691869091}

In [ ]:
import numpy as np

In [ ]:
def softmax(logits):
  return np.exp(logits) / np.sum(np.exp(logits), axis=0)

In [ ]:
preds = [np.argmax(softmax(logits)) for logits in model_outputs]

In [ ]:
label_to_code = dict(zip(train.labels, train.Code))

In [ ]:
test_df["preds"] = preds

In [ ]:
test_df["country"] = test.country

In [ ]:
test_df.head()

,text,labels,preds,country,true_domain,pred_domain
0,La ministra colombiana de Exteriores considera...,37,11,S,6,3
1,DEPORTES,31,31,S,5,5
2,É enorme a importância dessas atividades no de...,31,44,B,5,5
3,"Com o fim da guerra fria, iniciou-se um novo p...",5,3,B,1,1
4,Revelaram-se com inapagável nitidez novas teia...,13,13,P,3,3


In [ ]:
t = test_df[test_df.country == 'P']
f_macro(t.labels, t.preds)

0.38598591637028656

In [ ]:
domain_to_label = {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7:[], 9:[]}
for label, code in label_to_code.items():
  domain = code // 100
  domain_to_label[domain].append(label)

In [ ]:
preds_domain = []
for label in preds:
  for domain, labels in domain_to_label.items():
    if label in labels:
      preds_domain.append(domain)

In [ ]:
true_domain = []
for label in test_df.labels:
  for domain, labels in domain_to_label.items():
    if label in labels:
      true_domain.append(domain)

In [ ]:
test_df["true_domain"] = true_domain
test_df["pred_domain"] = preds_domain

In [ ]:
test_df.head()

,text,labels,preds,country,true_domain,pred_domain
0,La ministra colombiana de Exteriores considera...,37,11,S,6,3
1,DEPORTES,31,31,S,5,5
2,É enorme a importância dessas atividades no de...,31,44,B,5,7
3,"Com o fim da guerra fria, iniciou-se um novo p...",5,3,B,1,1
4,Revelaram-se com inapagável nitidez novas teia...,13,13,P,3,3


In [ ]:
t = test_df[test_df.country == 'P']

In [ ]:
#t = test_df[test_df.country == 'I']
t = test_df
f_macro(t.true_domain, t.pred_domain)

0.6692553009696277

In [ ]:
test_df.country.unique()

array(['S', 'B', 'P', 'I'], dtype=object)

In [ ]:
print("Domain Accuracy: {}".format(metrics.accuracy_score(true_domain, preds_domain)))
print("Domain F-Macro: {}".format(f_macro(true_domain, preds_domain)))
print("Domain F-Micro: {}".format(f_micro(true_domain, preds_domain)))
print("Domain MCC: {}".format(metrics.matthews_corrcoef(true_domain, preds_domain)))

Domain Accuracy: 0.7158686189157103
Domain F-Macro: 0.6684285966249962
Domain F-Micro: 0.7158686189157103
Domain MCC: 0.6551811783986897
